In [1]:
import gym
env = gym.make('gym_energyplus:EnergyPlus-v0')
observation = env.reset()
action = env.action_space.sample()
observation, reward, done, info = env.step(action)
print(action)
print(observation)
print(reward)
print(done)

Error: A module (gym_energyplus) was specified for the environment but was not found, make sure the package is installed with `pip install` before calling `gym.make()`

In [ ]:
!pip3 install https://download.pytorch.org/whl/cpu/torch-1.1.0-cp35-cp35m-linux_x86_64.whl

In [96]:
!pip install import_ipynb
import import_ipynb
import nn_lib
from nn_lib import * #MultiLayerNetwork, save_network, load_network

import gym
env = gym.make('gym_energyplus:EnergyPlus-v0')

import numpy as np
import random

#S + A -> R
class critic(MultiLayerNetwork):
    def __init__(self, env):
        action_dim = len(env.action_space.low)
        observation_dim = len(env.observation_space.low)
        
        neurons = [8, 1]
        activations = ["sigmoid", "sigmoid"]
        MultiLayerNetwork.__init__(self, action_dim+observation_dim, neurons, activations)
    '''  
    def forward(self, sa):
        action_dim = len(env.action_space.low)
        observation_dim = len(env.observation_space.low)
        x = sa.reshape((1, action_dim+observation_dim))
        y = MultiLayerNetwork.forward(self, x)
        return y[0]
    '''
#S -> a
class actor(MultiLayerNetwork):
    def __init__(self, env):
        action_dim = len(env.action_space.low)
        observation_dim = len(env.observation_space.low)
        
        neurons = [8, action_dim]
        activations = ["sigmoid", "sigmoid"]
        MultiLayerNetwork.__init__(self, observation_dim, neurons, activations) 
        
    def forward(self, s1):
        observation_dim = len(env.observation_space.low)
        x = s1.reshape((1, observation_dim))
        y = MultiLayerNetwork.forward(self, x)
        return y[0]*2 -1

def ddpg(env):
    action_dim = len(env.action_space.low)
    observation_dim = len(env.observation_space.low)
    M = 3 # episodes
    T = 10 # timestep
    N = 3 # mini batch size
    gamma = 0.9 # discounted factor
    tao = 0.5
    learning_rate = 0.01
    
    #random initialize critic network and actor network
    net_critic = critic(env)
    net_actor = actor(env)
    
    #initialize target network 
    target_Q = net_critic
    target_mu = net_actor
    
    #initialize replay buffer 
    R = []
    
    for episode in range(M):
        s1 = env.reset()
        for t in range(T):
            a1 = net_actor(s1) + np.random.normal(0, 1.0/(episode+1),1)
            s2, r1, done, info = env.step(a1)
            R.append([s1, a1, r1, s2])
            s1 = s2
            
            if len(R)>N:
                #sample a random mini-batch of N transitions from R
                minibatch = random.sample(R, N)
                
                #set y_i
                delta = np.zeros(action_dim)
                y = []
                #x = np.zeros(N,action_dim+observation_dim)
                x = []
                for i in range(N):
                    [s1, a1, r1, s2] = minibatch[i]
                    a2 = target_mu(s2)
                    y.append(r1 + gamma*target_Q(np.append(s2, a2))[0])
                    #x[i] = np.append(s1, a1)
                    x.append(np.append(s1, a1))
                    
                    a_1 = net_actor(s1)
                    o = net_critic(np.append(s1, a_1))[0]
                    delta += net_critic.backward([o])[0][-action_dim:]
                    net_critic.update_params(learning_rate)
                    
                delta = delta/N
                print(delta)
                
                #update critic
                trainer = Trainer(
                    network=net_critic,
                    batch_size=N,
                    nb_epoch=1,
                    learning_rate=0.01,
                    loss_fun="mse",
                    shuffle_flag=True,
                    )
                #print('-----')
                #print(np.asarray(x))
                #print(np.asarray(y))
                trainer.train(np.asarray(x), np.asarray(y))
                
                #update actor policy using the sampled policy gradient
                net_actor.backward(delta)
                net_actor.update_params(learning_rate)
                
                #update the target networks
                target_Q = con(net_critic, target_Q, tao)
                target_mu = con(net_actor, target_mu, tao)
                
                
        ## evaluation
        s = env.reset()
        loss = 0
        for t in range(20):
            a = net_actor(s)
            s, r, _, _ = env.step(a)
            loss+=r
        print(episode,':loss=',loss)
                
def con(net1, net2, tao):
    for i in range(len(net1._layers)):
        if i%2 == 0:
            net1._layers[i]._W = tao*net1._layers[i]._W + (1-tao)*net2._layers[i]._W
            net1._layers[i]._b = tao*net1._layers[i]._b + (1-tao)*net2._layers[i]._b
    return net1
    
ddpg(env)

0 ---activation: sigmoid
1 ---activation: sigmoid
0 ---activation: sigmoid
1 ---activation: sigmoid
Starting new environment
start_instance(): idx=0, model_file=/home/xiaonanchong/rl-testbed-for-energyplus/EnergyPlus/Model/2ZoneDataCenterHVAC_wEconomizer_Temp_Fan.idf
start_instance(): weather_files[0]=/usr/local/EnergyPlus-8-8-0/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw
Starting EnergyPlus with command: /usr/local/EnergyPlus-8-8-0/energyplus -r -x -d log/output/episode-00000000 -w log/output/episode-00000000/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw log/output/episode-00000000/2ZoneDataCenterHVAC_wEconomizer_Temp_Fan.idf
PipeIo.readline: Opening OBS pipe [/tmp/extctrl_152_obs]
PipeIo.writeline: Opened ACT pipe /tmp/extctrl_152_act
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
EnergyPlusEnv: Severe error(s) occurred. Error count: -1
EnergyPlusEnv: Check contents of log/output/episode-00000000/eplusout.err
Starting new